In [14]:
%%capture
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import time
from multiprocessing import Pool

from transformers import AutoTokenizer,AutoModelForSequenceClassification,TextClassificationPipeline,pipeline,AutoModel

In [2]:
# !pip install transformers

In [15]:
# load model, tokenizer
num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained("qlqqqk/ft_batch32_con_sen")

# # freeze layers
# for param in model.roberta.parameters():
#     param.requires_grad = True

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [16]:
# pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores=True, batch_size = 32, num_workers = 85) #device = device,
pipe  = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores = True)

/home/wjswps/.local/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


## 데이터 세팅
- lyric(score_test.tsv) : (1,6) 가사 벡터
- meta(lyric_data_v3.tsv) : 인덱스 기준 노래 제목/가수
- emotion: 사용자의 감정벡터


### emotion

In [17]:
input = '오늘도 나는 라임오렌지나무를 심는다.' # inference 문장

emotion = []
for i in pipe(input):
  print(i[0]['score']) #확인용
  emotion.append(i[0]['score'])
  emotion.append(i[1]['score'])
  emotion.append(i[2]['score'])
  emotion.append(i[3]['score'])
  emotion.append(i[4]['score'])
  emotion.append(i[5]['score'])
  

0.3230167031288147


In [18]:
emotion = np.array(emotion).reshape(1,-1)
emotion #{'기쁨':0,'긴장':1,'평화':2,'슬픔':3,'분노':4,'중립':5}

array([[0.3230167 , 0.09041135, 0.00680735, 0.0459486 , 0.10450783,
        0.42930818]])

### lyric

In [19]:
# index num이 lyric_data_v3와 매칭되는 번호 
lyric = pd.read_csv('data/score_test.tsv',sep='\t')
lyric = lyric.drop(columns=['Unnamed: 0','lyric'])
lyric.shape,lyric.head()

((112512, 6),
     LABEL_0   LABEL_1   LABEL_2   LABEL_3   LABEL_4   LABEL_5
 0  0.321668  0.050276  0.000602  0.013912  0.100803  0.512739
 1  0.211127  0.195028  0.000657  0.017706  0.143551  0.431931
 2  0.946268  0.002743  0.003444  0.031837  0.003763  0.011944
 3  0.003543  0.029312  0.000641  0.821336  0.136336  0.008832
 4  0.239259  0.642580  0.001295  0.056139  0.040510  0.020217)

In [20]:
ly_list = []
for i in lyric.values:
    ly_list.append(np.array(i).reshape(1,-1))
ly_list[27073]

array([[0.27414498, 0.18235692, 0.23758395, 0.13272549, 0.1603689 ,
        0.01281983]])

### meta

In [21]:
meta = pd.read_csv('data/lyric_data_v3.tsv',sep='\t')
meta.head()

,lyric,id,song_id,song_name,artist,genre,album,album_id,release
0,Feelings nothing more than feelings Trying to ...,0,4639910,Feelings,Various Artists,GN0900,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,20140512
1,i feel alright now but don't know how to speak...,8,1657318,Lovers’ Leap (Feat. Qypthone),애플스,GN0300,Sexualmetro,353020,20070625
2,그대 스치는 바람처럼 불어와서 내 곁에 머무른 사람 나도 몰래 내쉬는 숨처럼 익숙해...,9,30310140,"사랑, 그대라는 멜로디",진호,GN0100,Pastel Reflection,10047088,20170320
3,그냥 생각 없이 이렇다 할 뜻도 없이 쉼 없이 웃으며 떠드는 이들을 가만히 두리번거...,19,5579388,무얼 기다리나 (Feat. 조원선),이영훈,"GN0500,GN0800",내가 부른 그림 2,2303168,20150205
4,너를보면 나는 잠이와 이상하다 그치 잠이오면 나는 잠을자 이상하다 그치 자면서 너에...,29,3953129,창밖에 잠수교가 보인다,박영민,GN1500,창밖에 잠수교가 보인다 OST,2165909,19840910


## 코사인 유사도
- 2d shape error로 인해 각 벡터마다 reshape(1,-1)을 진행하였음.
- 유사도가 가장 높은 5개 출력하는 코드

In [22]:
cosine_sim = cosine_similarity(emotion,ly_list[0])
cosine_sim

array([[0.99102589]])

In [23]:
''''기쁨':0,'긴장':1,'평화':2,'슬픔':3,'분노':4,'중립':5
코사인 유사도의 순서대로 5개 출력'''
cosine = []
n= 0
for i,j in enumerate(ly_list):
  cosine_sim = cosine_similarity(emotion,j)
  cosine.append((i,cosine_sim))
resys = sorted(cosine,key=lambda x:x[1],reverse=True)[:5]
print(resys)
# sorted_similar_music = sorted(cosine_sim,key=lambda x:x[1],reverse=True)[1:]
# print(np.argsort(cosine_sim[0]))

[(71731, array([[0.99905687]])), (35552, array([[0.99840071]])), (103689, array([[0.99827305]])), (12894, array([[0.99824959]])), (73987, array([[0.99799773]]))]


In [48]:
for i in resys:
    n = i[0]
    artist = meta.iloc[n]['artist']
    song = meta.iloc[n]['song_name']
    ly_vec = ly_list[n]
    print(f'가수:{artist} 곡:{song} 벡터값{ly_vec}')

가수:윤항기 곡:나는 행복합니다 벡터값[[0.33905092 0.10154315 0.00080187 0.03704604 0.10188568 0.41967234]]
가수:제8극장 곡:다섯번째 알람 소리에 벡터값[[0.33096766 0.09356253 0.0010026  0.01770524 0.1187799  0.43798208]]
가수:ABBA 곡:Ring Ring 벡터값[[0.35185727 0.09457599 0.00127529 0.03336093 0.08668768 0.43224278]]
가수:이미자 곡:떠난 사람 벡터값[[0.33758652 0.093244   0.00063492 0.01421414 0.10880288 0.44551751]]
가수:클랑 (KLANG) 곡:Horizon 벡터값[[0.3510071  0.09588897 0.00079485 0.0200015  0.10299969 0.42930791]]


In [ ]:

# def weighted_sum(a,b,vec):
# # 예시 곡네임 분리
#     fin_vec = np.empty((0,6))
#     for i in range(len(vec)):
#         ly_name = vec[i][0]

#         #가사벡터와 멜로디 벡터 분리
#         lyrics = vec[i][1:7]
#         mel = vec[i][7:]

#         wei_vec = a*lyrics + b*mel
#         # wei_vec = wei_vec.reshape(1,6)
#         fin_vec = np.vstack((fin_vec,wei_vec))
#     return ly_name,fin_vec
# name, vectors = weighted_sum(a,b,vec2)

In [ ]:
# cosine_sim = cosine_similarity(vec1,vectors)
# print(cosine_sim)
# print(sorted(cosine_sim[0],reverse=True)[:5]) # 나중에는 원본백터(곡 이름) 출력되도록
# # sorted_similar_music = sorted(cosine_sim,key=lambda x:x[1],reverse=True)[1:]
# print(np.argsort(cosine_sim[0]))